# net_collector — Jupyter 使用例

踏み台サーバ経由でネットワーク機器にコマンドを実行し、結果を確認・差分表示するノートブックです。

## 1. セットアップ

In [1]:
import sys
import logging
from pathlib import Path

# プロジェクトルートを sys.path に追加
PROJECT_ROOT = Path(".").resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# ロギング設定（INFO レベル以上を表示）
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler("execution.log", encoding="utf-8"),
    ],
)

from src import NetCollector
print("セットアップ完了")

セットアップ完了

## 2. NetCollector の初期化

In [2]:
# 設定ファイルを読み込む
nc = NetCollector(
    hosts_file="configs/hosts.yaml",
    commands_file="configs/commands.yaml",
    review_file="configs/review_points.yaml",
)

print("=== 踏み台サーバ ===")
for b in nc.hosts_config.bastions:
    print(f"  {b.name}: {b.protocol}://{b.host}:{b.port}")

print("\n=== 機器 ===")
for d in nc.hosts_config.devices:
    print(f"  {d.name}: [{d.vendor}] {d.protocol}://{d.host}:{d.port}")

print("\n=== コマンド ===")
for c in nc.commands_config.commands:
    print(f"  [{c.vendor or 'all'}] {c.name}: {c.command}")

=== 踏み台サーバ ===
  bastion-1: telnet://192.168.3.23:23

=== 機器 ===
  vmx1: [juniper] ssh://172.20.20.2:22

=== コマンド ===
  [juniper] show_version: show version
  [juniper] show_interfaces: show interfaces terse
  [juniper] show_route: show route
  [juniper] show_bgp_summary: show bgp summary

## 3. コマンド実行

指定した機器に接続し、コマンドを実行します。  
実行前に Yes/No の確認プロンプトが表示されます。

In [8]:
# vmx1 に接続してコマンドを実行
result = nc.run(device="vmx1", confirm=True)

if result:
    print(f"\n実行完了: タイムスタンプ = {result.timestamp}")


機器 [vmx1] (172.20.20.2) に 4 件のコマンドを実行しますか? [y/N]:  y
2026-02-24 11:17:39,381 [INFO] src: 接続開始: vmx1 (172.20.20.2) via 1 踏み台
2026-02-24 11:17:41,514 [INFO] src.executor: [vmx1] コマンド実行 [1/4]: show version2026-02-24 11:17:39,381 [INFO] src: 接続開始: vmx1 (172.20.20.2) via 1 踏み台
2026-02-24 11:17:41,514 [INFO] src.executor: [vmx1] コマンド実行 [1/4]: show version
2026-02-24 11:17:41,976 [INFO] src.executor: [vmx1] コマンド実行 [2/4]: show interfaces terse2026-02-24 11:17:39,381 [INFO] src: 接続開始: vmx1 (172.20.20.2) via 1 踏み台
2026-02-24 11:17:41,514 [INFO] src.executor: [vmx1] コマンド実行 [1/4]: show version
2026-02-24 11:17:41,976 [INFO] src.executor: [vmx1] コマンド実行 [2/4]: show interfaces terse
2026-02-24 11:17:42,376 [INFO] src.executor: [vmx1] コマンド実行 [3/4]: show route2026-02-24 11:17:39,381 [INFO] src: 接続開始: vmx1 (172.20.20.2) via 1 踏み台
2026-02-24 11:17:41,514 [INFO] src.executor: [vmx1] コマンド実行 [1/4]: show version
2026-02-24 11:17:41,976 [INFO] src.executor: [vmx1] コマンド実行 [2/4]: show interfaces terse
2026-02-

2026-02-24 11:17:39,381 [INFO] src: 接続開始: vmx1 (172.20.20.2) via 1 踏み台2026-02-24 11:17:41,514 [INFO] src.executor: [vmx1] コマンド実行 [1/4]: show version2026-02-24 11:17:41,976 [INFO] src.executor: [vmx1] コマンド実行 [2/4]: show interfaces terse2026-02-24 11:17:42,376 [INFO] src.executor: [vmx1] コマンド実行 [3/4]: show route2026-02-24 11:17:42,708 [INFO] src.executor: [vmx1] コマンド実行 [4/4]: show bgp summary2026-02-24 11:17:43,035 [INFO] src: 保存完了: outputs/vmx1/20260224_111743/
実行完了: タイムスタンプ = 20260224_111743

## 4. 結果の表示

キーワードハイライト付きで結果を表示します。

In [5]:
if result:
    result.show()

## 5. 保存済みタイムスタンプの確認

In [6]:
from src.output import OutputSaver

saver = OutputSaver("outputs")
timestamps = saver.list_timestamps("vmx1")

print("保存済みタイムスタンプ（vmx1）:")
for ts in timestamps:
    print(f"  {ts}")

保存済みタイムスタンプ（vmx1）:
  20260224_101812
  20260224_101825
  20260224_103619
  20260224_104244
  20260224_105207
  20260224_105909
  20260224_110152
  20260224_110506
  20260224_110518
  20260224_110551
  20260224_111139
  20260224_111429
  20260224_111459

## 6. 差分表示

過去の保存データとの差分を表示します。  
`old_timestamp` を省略すると直前の保存データと比較します。

In [7]:
if result:
    # 直前の保存データと比較
    result.diff()

    # 特定タイムスタンプと比較する場合
    # result.diff(old_timestamp="20260223_100000")

比較: 20260224_111429  →  20260224_111459